### **Installation**

In [ ]:
%%capture
!pip install open_spiel

##### **Parameters / Documentation**

Documentation is a bit limited for the poker_spiel - parameters are available via [universal_poker.cc](https://github.com/google-deepmind/open_spiel/blob/master/open_spiel/games/universal_poker/universal_poker.cc)
```
{
  // "numPlayers" is stored as GameParameter(2) -> INT
  {"numPlayers", GameParameter(2)},

  // "betting" is GameParameter(std::string("nolimit")) -> STRING
  {"betting", GameParameter(std::string("nolimit"))},

  // "stack" is GameParameter(std::string("1200 1200")) -> STRING
  {"stack", GameParameter(std::string("1200 1200"))},

  // "blind" is GameParameter(std::string("100 100")) -> STRING
  {"blind", GameParameter(std::string("100 100"))},

  // "raiseSize" is GameParameter(std::string("100 100")) -> STRING
  {"raiseSize", GameParameter(std::string("100 100"))},

  // "numRounds" is GameParameter(2) -> INT
  {"numRounds", GameParameter(2)},

  // "firstPlayer" is GameParameter(std::string("1 1")) -> STRING
  {"firstPlayer", GameParameter(std::string("1 1"))},

  // "maxRaises" is GameParameter(std::string("")) -> STRING (empty by default)
  {"maxRaises", GameParameter(std::string(""))},

  // "numSuits" is GameParameter(4) -> INT
  {"numSuits", GameParameter(4)},

  // "numRanks" is GameParameter(6) -> INT
  {"numRanks", GameParameter(6)},

  // "numHoleCards" is GameParameter(1) -> INT
  {"numHoleCards", GameParameter(1)},

  // "numBoardCards" is GameParameter(std::string("0 1")) -> STRING
  {"numBoardCards", GameParameter(std::string("0 1"))},

  // "bettingAbstraction" is GameParameter(std::string("fcpa")) -> STRING
  {"bettingAbstraction", GameParameter(std::string("fcpa"))},

  // "potSize" is GameParameter(0) -> INT
  {"potSize", GameParameter(0)},

  // "boardCards" is GameParameter("") -> STRING
  {"boardCards", GameParameter("")},

  // "handReaches" is GameParameter("") -> STRING
  {"handReaches", GameParameter("")},
}
```

### **Pokerbot Example**

**Imports**

In [ ]:
import pyspiel
import random

**Helper Code**

In [ ]:
RANK_CHARS = ["2","3","4","5","6","7","8","9","T","J","Q","K","A"]
SUIT_CHARS = ["c","d","h","s"]  # clubs, diamonds, hearts, spades

def decode_card_id(card_id):
    """
    Convert integer card_id (0..51) to a string like '2c', 'Ah', etc.
    rank = card_id >> 2, suit = card_id & 3
    """
    rank = card_id >> 2
    suit = card_id & 3
    return RANK_CHARS[rank] + SUIT_CHARS[suit]

def describe_action(a):
    if a == 0: return "Fold"
    elif a == 1: return "Call/Check"
    elif a == 2: return "Bet/Raise"
    elif a == 3: return "All-In"
    elif a == 4: return "Half-Pot"
    return f"CustomBet={a}"

# internal parser (ignore this -- this is to make making the bots easier)
def parse_state_cards(state_str, num_players=2):
    """
    Returns a dict with:
      {
        'player0': ['Qc','Jc'],  # example hole cards
        'player1': ['2d','5h'],  # ...
        'board': ['As','Kh','Td']
      }
    for each "P{i} Cards: [..][..]" line, we parse the bracketed tokens
    also parse "BoardCards [..][..][..]" if present
    """
    result = {f"player{i}":[] for i in range(num_players)}
    result["board"] = []

    lines = state_str.split("\n")
    for line in lines:
        line = line.strip()
        # e.g. "P0 Cards: [Qc][Jc]"
        if line.startswith("P") and "Cards:" in line:
            # find which player
            # e.g. "P0 Cards:" => '0'
            prefix, after = line.split(":", 1)  # ["P0 Cards", " [Qc][Jc]"]
            player_id = prefix.strip()[1]  # e.g. "0"
            cards = []
            # parse bracket tokens
            for chunk in after.strip().split("]"):
                chunk = chunk.strip()
                if chunk.startswith("["):
                    card = chunk[1:].strip()  # e.g. "Qc"
                    if card:
                        cards.append(card)
            result[f"player{player_id}"] = cards
        elif line.startswith("BoardCards"):
            # e.g. "BoardCards [As][Kh][Td]"
            after = line.split("BoardCards")[-1]
            # parse similarly
            cards = []
            for chunk in after.strip().split("]"):
                chunk = chunk.strip()
                if chunk.startswith("["):
                    card = chunk[1:].strip()
                    if card:
                        cards.append(card)
            result["board"] = cards
    return result

#### **Main Code**

Base Pokerbot Class

In [ ]:
class PokerBot:
    """
    Base class for any bot.
    - 'name' is for display.
    - You must override 'get_action(state)'.
    """
    def __init__(self, name="Bot"):
        self.name = name

    def get_action(self, state):
        """Return an int representing your action choice (fold=0, call=1, etc.). check the util cell above for more info / references"""
        raise NotImplementedError

**Creating a Pokerbot - Examples**

All actions:


In [ ]:
class AlwaysCallBot(PokerBot):
    def get_action(self, state):
        acts = state.legal_actions()
        if 1 in acts:
            return 1
        else:
            acts[0]

In [ ]:
class RaiseBot(PokerBot):
    """Raise if possible, else call, else fold."""
    def get_action(self, state):
        acts = state.legal_actions()
        if 2 in acts: return 2
        elif 1 in acts: return 1
        else: return acts[0]

In [ ]:
# --------------- EXAMPLE HEURISTIC BOT ---------------
class HeuristicBot(PokerBot):
    """
    Example: If the board is [As][Kh][Td] and we hold [Qc][Jc],
    we do a naive conditional prob check (p(A|B) ~ 15%).
    If that prob is small, we raise/all-in; otherwise, we call.
    Else (different scenario), default to calling.

    This approach uses naive string matching on 'state.to_string()':
       has_our_flop = "BoardCards [As][Kh][Td]" in s
       has_our_hand = "Qc][Jc" in s
    In reality, you'd parse the board and hole cards more robustly.
    """
    def get_action(self, state):
        s = state.to_string()
        acts = state.legal_actions()

        has_our_flop = "BoardCards [As][Kh][Td]" in s
        has_our_hand = "Qc][Jc" in s
        if has_our_flop and has_our_hand:
            # Check if opponent likely raised preflop (look for 'p' in action sequence)
            seq_part = s.split("Action Sequence:")[-1]
            if 'p' in seq_part:
                # p(A|B) ~ 0.15
                pAB = 0.8 * (2/52) / 0.2
                if pAB < 0.2:
                    if 3 in acts: return 3
                    if 2 in acts: return 2
                    if 1 in acts: return 1
                    return acts[0]
                else:
                    return 1 if 1 in acts else acts[0]
            else:
                if 2 in acts: return 2
                elif 1 in acts: return 1
                else: return acts[0]
        else:
            # Default strategy: call
            return 1 if 1 in acts else acts[0]

In [ ]:
#@title Code for running the game - expand if you're interested, but it's essentially a parser and setting parameters
def run_match(bot0, bot1, num_hands=3, game_params=None, verbose=True, num_players=2):
    """
    Plays 'num_hands' of 2-player No-Limit Texas Hold'em between 'bot0' & 'bot1'.
    Final results are printed after all hands.
    """
    if game_params is None:
        game_params = {
            "betting": "nolimit",
            "numPlayers": num_players,
            "blind": "1 2"
        }
    game = pyspiel.load_game("universal_poker", game_params)
    totals = [0.0, 0.0]

    for h in range(num_hands):
        if verbose:
            print("\n=== HAND", h+1, "===")
        state = game.new_initial_state()

        while not state.is_terminal():
            if state.is_chance_node():
                outs = state.chance_outcomes()
                actions = []
                probs = []
                for pair in outs:
                    actions.append(pair[0])
                    probs.append(pair[1])
                pick = random.choices(actions, probs)[0]
                state.apply_action(pick)
                if verbose:
                    card_desc = decode_card(pick)
                    print("Chance dealt card =>", card_desc)
            else:
                cp = state.current_player()
                if cp == 0:
                    choice = bot0.get_action(state)
                else:
                    choice = bot1.get_action(state)

                if verbose:
                    print("Player", cp, "(", [bot0.name, bot1.name][cp], "):", describe_action(choice))
                    print(state)

                state.apply_action(choice)

        r = state.returns()
        totals[0] += r[0]
        totals[1] += r[1]
        if verbose:
            print("Hand result ->", bot0.name, ":", r[0], ",", bot1.name, ":", r[1])

    print("\n*** MATCH COMPLETE ***")
    print("Total for", bot0.name, ":", totals[0])
    print("Total for", bot1.name, ":", totals[1])
    print("**********************")

def run_round_robin(bots, num_hands=3, game_params=None, verbose=False):
    """
    Each pair of bots plays a match.
    Returns final scoreboard in dict form.
    Also prints a summary in a nice format.
    """
    if game_params is None:
        game_params = {
            "betting": "nolimit",
            "numPlayers": 2,
            "blind": "1 2"
        }

    # track cumulative scores
    scores = {}
    for b in bots:
        scores[b.name] = 0.0

    # run each pair
    num_bots = len(bots)
    for i in range(num_bots):
        for j in range(i+1, num_bots):
            botA = bots[i]
            botB = bots[j]
            print("\n============================")
            print("MATCH:", botA.name, "vs.", botB.name)
            print("============================")

            # We'll do a simpler in-code version of run_match to accumulate results
            game = pyspiel.load_game("universal_poker", game_params)
            match_totals = [0.0, 0.0]

            for h in range(num_hands):
                if verbose:
                    print("\n=== HAND", h+1, "===")
                state = game.new_initial_state()

                while not state.is_terminal():
                    if state.is_chance_node():
                        outs = state.chance_outcomes()
                        action_list = []
                        prob_list = []
                        for item in outs:
                            action_list.append(item[0])
                            prob_list.append(item[1])
                        pick = random.choices(action_list, prob_list)[0]
                        state.apply_action(pick)
                        if verbose:
                            print("Chance -> dealt:", decode_card(pick))
                    else:
                        cp = state.current_player()
                        if cp == 0:
                            chosen = botA.get_action(state)
                            if verbose:
                                print(botA.name, ":", describe_action(chosen))
                        else:
                            chosen = botB.get_action(state)
                            if verbose:
                                print(botB.name, ":", describe_action(chosen))
                        state.apply_action(chosen)
                        if verbose:
                            print(state)

                returns = state.returns()
                match_totals[0] += returns[0]
                match_totals[1] += returns[1]
                if verbose:
                    print("Hand result ->", botA.name, ":", returns[0], ",", botB.name, ":", returns[1])

            # after match
            print("Final match totals ->", botA.name, ":", match_totals[0], ",", botB.name, ":", match_totals[1])
            scores[botA.name] += match_totals[0]
            scores[botB.name] += match_totals[1]

    print("\n==== ROUND ROBIN RESULTS ====")
    # Print scoreboard in a nice format
    sorted_names = sorted(scores.keys())
    for nm in sorted_names:
        val = scores[nm]
        print(f"{nm} => {val}")
    return scores

Usage:
```
run_match(bot1, bot2, num_hands, verbose)
```
* bot1/bot2 = bots
* num_hands = number of hands to play
* verbose = debugger if you want to see all the hands played

```
run_round_robin(bots, num_hands=3, game_params=None, verbose=False)
```

random example bots

a bot that always folds

In [ ]:
class FoldBot(PokerBot):
    """Always folds if it can, else picks the first available action."""
    def get_action(self, state):
        legal = state.legal_actions()
        if 0 in legal:
            return 0  # fold
        else:
            return legal[0]

In [ ]:
class CallBot(PokerBot):
    """Always calls/checks if possible, else folds."""
    def get_action(self, state):
        legal = state.legal_actions()
        if 1 in legal:
            return 1  # call/check
        else:
            return legal[0]  # fallback (often fold)

In [ ]:
class RaiseBot(PokerBot):
    """Always raises if possible, else calls, else folds."""
    def get_action(self, state):
        legal = state.legal_actions()
        if 2 in legal:
            return 2  # bet/raise
        elif 1 in legal:
            return 1  # call
        else:
            return legal[0]  # fold

In [ ]:
class RandomBot(PokerBot):
    """Chooses any legal action uniformly at random."""
    def get_action(self, state):
        legal = state.legal_actions()
        return random.choice(legal)

In [ ]:
class MinRaiseBot(PokerBot):
    """
    Always tries the minimum raise above a call,
    if that is supported in fullgame mode.
    Otherwise calls, otherwise folds.
    """
    def get_action(self, state):
        legal = state.legal_actions()
        if 2 in legal:
            return 2
        elif 1 in legal:
            return 1
        else:
            return legal[0]

More complex examples

In [ ]:
class LooseBot(PokerBot):
    """
    Loose style: tries to raise 70% if it's legal, otherwise calls, rarely folds.
    """
    def get_action(self, state):
        legal = state.legal_actions()
        can_raise = False
        for act in legal:
            if act == 2:
                can_raise = True
                break

        if can_raise:
            # 70% chance to raise, else call
            val = random.random()
            if val < 0.7:
                return 2
            else:
                if 1 in legal:
                    return 1
                else:
                    return legal[0]
        else:
            if 1 in legal:
                return 1
            else:
                return legal[0]

In [ ]:
class Aggressive(PokerBot):
    """
    If it doesn't see an 'A' or 'K' in its hole cards, it folds preflop.
    Otherwise calls or raises. Very naive logic.
    """
    def get_action(self, state):
        legal = state.legal_actions()
        s = state.to_string()
        # find which player
        current_player = state.current_player()
        hole_line = "P" + str(current_player) + " Cards:"
        # parse line
        cards_found = []
        lines = s.split("\n")
        for line in lines:
            if line.startswith(hole_line):
                # e.g. "P0 Cards: [4c][Ac]"
                parts = line.split("]")
                for p in parts:
                    p = p.strip()
                    if p.startswith("["):
                        c = p[1:]  # e.g. "4c" or "Ac"
                        cards_found.append(c)
                break

        # if none of our hole cards is K or A => fold if possible
        rank_trigger = False
        for c in cards_found:
            # first char is rank
            rank_char = c[0]
            if rank_char == "K" or rank_char == "A":
                rank_trigger = True

        if not rank_trigger:
            # try to fold
            if 0 in legal:
                return 0
            else:
                if 1 in legal:
                    return 1
                else:
                    return legal[0]
        else:
            # if we can raise, do it
            if 2 in legal:
                return 2
            elif 1 in legal:
                return 1
            else:
                return legal[0]

In [ ]:
class ConditionalProbBot(PokerBot):
    """
    Toy scenario-based logic from lecture example: if we see the flop [As][Kh][Td] and hole [Qc][Jc],
    we do a naive p(A|B) ~ 15% check -> raise or call. Otherwise, just call.
    """
    def get_action(self, state):
        legal = state.legal_actions()
        s = state.to_string()
        if "BoardCards [As][Kh][Td]" in s and "Qc][Jc" in s:
            # check if there's a 'p' in the action seq => interpret as preflop raise
            seq_part = s.split("Action Sequence:")[-1]
            if 'p' in seq_part:
                # ~15% chance
                pAB = 0.8*(2.0/52.0)/0.2
                if pAB < 0.2:
                    if 3 in legal:
                        return 3
                    if 2 in legal:
                        return 2
                    if 1 in legal:
                        return 1
                    return legal[0]
                else:
                    if 1 in legal:
                        return 1
                    return legal[0]
            else:
                if 2 in legal:
                    return 2
                elif 1 in legal:
                    return 1
                return legal[0]
        else:
            if 1 in legal:
                return 1
            else:
                return legal[0]

In [ ]:
class AllIn(PokerBot):
    """
    go all in all the time

    pseudocode:
        check for legal actions
        check if you can all in
        else if check
        else do whatever is available in legal actions
    """
    def get_action(self, state):
      legal = state.legal_actions()
      can_allin = False
      for act in legal:
        if act == 3:
          can_allin = True
          break

      if can_allin:
        return 3
      elif 1 in legal:
        return 1
      else:
        return legal[0]

A more robust example

In [ ]:
class StrategicBot(PokerBot):
    """Makes strategic decisions based on hand strength, board cards, and heuristics."""
    def __init__(self, name="StrategicBot"):
        super().__init__(name)

    def evaluate_hand_strength(self, hole_cards, board_cards):
        """
        Simplistic hand evaluator.
        Returns: "strong", "decent", or "weak".
        """
        # Example: check for pairs or straight draws
        ranks = [card[0] for card in hole_cards + board_cards]
        unique_ranks = set(ranks)

        # Simplistic rules for now
        if len(unique_ranks) <= len(hole_cards):  # Pair or better
            return "strong"
        elif len(unique_ranks) <= len(hole_cards) + 1:  # Potential draw
            return "decent"
        else:
            return "weak"

    def get_action(self, state):
        # Get legal actions
        legal_actions = state.legal_actions()
        # Parse hole cards and board cards
        s = state.to_string()
        lines = s.split("\n")
        hole_cards = []
        board_cards = []
        current_player = state.current_player()

        for line in lines:
            if line.startswith(f"P{current_player} Cards:"):
                # Extract hole cards (e.g., "P0 Cards: [Ah][Kd]")
                parts = line.split("]")
                for p in parts:
                    if "[" in p:
                        hole_cards.append(p.split("[")[1])
            if line.startswith("BoardCards"):
                # Extract board cards (e.g., "BoardCards [As][Kh][Td]")
                parts = line.split("]")
                for p in parts:
                    if "[" in p:
                        board_cards.append(p.split("[")[1])

        # Evaluate hand strength
        hand_strength = self.evaluate_hand_strength(hole_cards, board_cards)

        # Decision-making
        if hand_strength == "strong":
            if 2 in legal_actions:  # Raise
                return 2
            elif 1 in legal_actions:  # Call/Check
                return 1
            else:  # Fallback (fold unlikely here)
                return legal_actions[0]
        elif hand_strength == "decent":
            if 1 in legal_actions:  # Call/Check
                return 1
            else:
                return legal_actions[0]  # Fallback (fold)
        else:  # Weak hand
            if 0 in legal_actions:  # Fold
                return 0
            elif 1 in legal_actions:  # Call/Check (rare)
                return 1
            else:
                return legal_actions[0]

In [ ]:
import numpy as np
import copy
import math

class MonteCarloNode():
  def __init__(self, state, parent=None):
    self.state = state
    self.win_amt = 0
    self.num_visits = 0
    self.parent = parent
    self.children = [] # (node, action)
    self.untried_actions = state.legal_actions()
    self.is_terminal = state.is_terminal()

class MonteCarloBot(PokerBot):
  ITERATION_BUDGET = 5000 # feel free to modify, higher budget = more simulations = better bot

  # from strategic bot
  def evaluate_hand_strength(self, state):
        hole_cards = []
        board_cards = []
        current_player = state.current_player()

        s = state.to_string()
        lines = s.split("\n")
        for line in lines:
            if line.startswith(f"P{current_player} Cards:"):
                # Extract hole cards (e.g., "P0 Cards: [Ah][Kd]")
                parts = line.split("]")
                for p in parts:
                    if "[" in p:
                        hole_cards.append(p.split("[")[1])
            if line.startswith("BoardCards"):
                # Extract board cards (e.g., "BoardCards [As][Kh][Td]")
                parts = line.split("]")
                for p in parts:
                    if "[" in p:
                        board_cards.append(p.split("[")[1])

        ranks = [card[0] for card in hole_cards + board_cards]
        unique_ranks = set(ranks)
        if len(unique_ranks) <= len(hole_cards):  # Pair or better
            return "strong"
        elif len(unique_ranks) <= len(hole_cards) + 1:  # Potential draw
            return "decent"
        else:
            return "weak"

  # from strategic bot
  def strategic_move(self, hand_strength, legal_actions):
        if hand_strength == "strong":
            return 2 if 2 in legal_actions else (1 if 1 in legal_actions else legal_actions[0])
        elif hand_strength == "decent":
            return 1 if 1 in legal_actions else legal_actions[0]
        else:
            return 0 if 0 in legal_actions else (1 if 1 in legal_actions else legal_actions[0])

  def get_action(self, state):
    self.root = MonteCarloNode(state)
    iters = 0
    while (iters < self.ITERATION_BUDGET):
      node = self.select(self.root)
      reward = self.simulate(node)
      self.backpropagate(node, reward)
      iters += 1
    action = self.best_child(self.root, 0)[1]
    return action

  def select(self, node):
    while not node.is_terminal:
        if len(node.untried_actions) != 0:
            return self.expand(node)
        else:
            node = self.best_child(node, c=1)[0]
    return node

  def expand(self, node):
    action = node.untried_actions.pop(0)
    child_state = copy.deepcopy(node.state)
    child_state.apply_action(action)
    child_node = MonteCarloNode(child_state, node)
    node.children.append((action, child_node))
    return child_node

  def simulate(self, node): # default policy: strong/weak hands
    cur_state = copy.deepcopy(node.state)
    while not cur_state.is_terminal():
      # evaluate the strength of your current hand
      hand_strength = self.evaluate_hand_strength(cur_state)
      legal_actions = cur_state.legal_actions()
      # make moves based on strategic bot
      action = self.strategic_move(hand_strength, legal_actions)
      cur_state.apply_action(action)
    return cur_state.returns()[0]

  def backpropagate(self, node, reward):
    # each node should store the number of wins for the player of its parent node
    while node is not None:
          node.num_visits += 1
          # reward = -reward
          node.win_amt -= reward
          node = node.parent

  def best_child(self, node, c=1):
    # determine the best child and action by applying the UCB formula
    best_child_node = None # to store the child node with best UCB
    best_action = None # to store the action that leads to the best child

    N_parent = node.num_visits
    best_ucb = 0

    for child in node.children:
        N_child = child[1].num_visits
        Q_child = child[1].win_amt
        cur_ucb = (Q_child / N_child) + (c * math.sqrt(2 * math.log(N_parent) / N_child))
        if cur_ucb > best_ucb or best_ucb == 0:
            best_ucb = cur_ucb
            best_action = child[0]
            best_child_node = child[1]

    return best_child_node, best_action


### Running Matches

In [ ]:
# bots

b1 = FoldBot("Folder")
b2 = CallBot("Caller")
b3 = RaiseBot("Raiser")
b4 = ConditionalProbBot("Conditional")
b5 = Aggressive("Aggro")
b6 = MonteCarloBot("MonteCarlo")
robust = StrategicBot("Strategy")

In [ ]:
run_match(b3, b6, num_hands=1, verbose=False)


*** MATCH COMPLETE ***
Total for Raiser : -1200.0
Total for MonteCarlo : 1200.0
**********************


In [ ]:
run_match(b3, b6, num_hands=10, verbose=False)


*** MATCH COMPLETE ***
Total for Raiser : -2392.0
Total for MonteCarlo : 2392.0
**********************


In [ ]:
run_round_robin([b1,b2,b3,b4,b5,b6, robust], num_hands=5, verbose=False)


MATCH: Folder vs. Caller
Final match totals -> Folder : -5.0 , Caller : 5.0

MATCH: Folder vs. Raiser
Final match totals -> Folder : -5.0 , Raiser : 5.0

MATCH: Folder vs. Conditional
Final match totals -> Folder : -5.0 , Conditional : 5.0

MATCH: Folder vs. Aggro
Final match totals -> Folder : -5.0 , Aggro : 5.0

MATCH: Folder vs. MonteCarlo
Final match totals -> Folder : -5.0 , MonteCarlo : 5.0

MATCH: Folder vs. Strategy
Final match totals -> Folder : -5.0 , Strategy : 5.0

MATCH: Caller vs. Raiser
Final match totals -> Caller : 18.0 , Raiser : -18.0

MATCH: Caller vs. Conditional
Final match totals -> Caller : 2.0 , Conditional : -2.0

MATCH: Caller vs. Aggro
Final match totals -> Caller : -6.0 , Aggro : 6.0

MATCH: Caller vs. MonteCarlo
Final match totals -> Caller : 6.0 , MonteCarlo : -6.0

MATCH: Caller vs. Strategy
Final match totals -> Caller : -72.0 , Strategy : 72.0

MATCH: Raiser vs. Conditional
Final match totals -> Raiser : 18.0 , Conditional : -18.0

MATCH: Raiser vs. A

{'Folder': -30.0,
 'Caller': -47.0,
 'Raiser': -1665.0,
 'Conditional': -1215.0,
 'Aggro': -3.0,
 'MonteCarlo': 3846.0,
 'Strategy': -886.0}